# Computing huge subspaces of diagonal harmonic polynomials:<br>Symmetries to the rescue!

<br>
<center>
  <large>[Nicolas M. Thiéry](http://Nicolas.Thiery.name)</large><br>
  [Laboratoire de Recherche en Informatique](http://lri.fr)<br>
  Université Paris Sud<br><br>
  
     
  Algorithms and Discrete Structures pole seminar<br>
  IRIF, 16 janvier 2018<br><br>
  
  [git repo with code,...](https://github.com/nthiery/harmonic-modules)
</center>

## Abstract

Last spring, I visited François Bergeron and we worked on his
favorite objects: the spaces H(n,k) of diagonal harmonic polynomials
in k sets of n variables. Those spaces connect to many areas of
algebraic combinatorics, representation theory and beyond, and the
case H(n,2) became famous a decade ago with the n! conjecture and
its proof by Haiman.

To fuel his ongoing studies François needed to compute the structure
of H(5,6). This is a space of dimension 6.10^5 made of polynomials
in 30 variables of degree up to 15, each having thousands of terms.

In this talk, I'll explain how the calculation can now be completed
in 45 minutes with a dozen cores and ~15Go of memory. This exploits
a combination of strategies (symmetries, representation theory of
the symmetric and general linear group, ...), each of which reduces
the complexity in time and memory by one or two orders of magnitude.

There will be little prerequisites and it's my hope that some
strategies (and maybe the code!) could be used in other contexts.

LaTeX definitions
$\def\QQ{\mathbb{Q}}$
$\def\NN{\mathbb{N}}$
$\def\x{\mathbf{x}}$
$\def\div{\operatorname{div}}$
$\def\harm{\operatorname{Harm}}$

## Motivations

<center>
    <img src="figures/crm.jpg"/ width="65%">
    <img src="figures/uqam.jpg"/ width="65%">
</center>

## Joint work with François Bergeron
<center><img src="figures/Francois.jpg" height="50%"></center>

## One of François's favorite objects
<img src="figures/book.jpg" style="float:right;"/>

The space $\harm(n,r)$ of **diagonal harmonic polynomials** in
$$\QQ\begin{bmatrix}
        x_{11}&\cdots&x_{1n}\\
        \vdots & & \vdots\\
        x_{r1}&\cdots&x_{rn}\\
     \end{bmatrix}$$
- $\harm(n,1)$ : dim=$n!$<br>
  graded character: Hall-Littlewood Polynomials<br>
  connections with invariant theory, ...
  <br><br>

- $\harm(n,2)$ : dim = $(n+1)^{n-1}$<br>
  graded character: Macdonald polynomials<br>
  $n!$-conjecture of Garsia/Bergeron<br>
  proof by Mark Haiman using the Hilbert scheme of points<br>
  <br><br>

- $\harm(n,r)$ : dim = ???<br>

## Project: pushing further the computer exploration

**Aim**: dimension and graded character of $\harm(6,5)$

Brute force linear algebra?
- polynomials in 30 variables of degree up to 15, with thousands of terms
- dimension: $\approx$ 3.7 million

In [1]:
d = 3.7 * 10^6
t = d^3 / 10.^9 * units.time.second
t.convert(units.time.year)

1606.19609335363*year

In [2]:
var('z');
f = ( 1/(1-z)^30 ).series(z,16); f # the graded dimension of the polynomial ring Q[x_ij]

1 + 30*z + 465*z^2 + 4960*z^3 + 40920*z^4 + 278256*z^5 + 1623160*z^6 + 8347680*z^7 + 38608020*z^8 + 163011640*z^9 + 635745396*z^10 + 2311801440*z^11 + 7898654920*z^12 + 25518731280*z^13 + 78378960360*z^14 + 229911617056*z^15 + Order(z^16)

In [3]:
D = sum(f.coefficients(sparse=False))
t = d^1.8 * D / 10.^9 * units.time.second
t.convert(units.time.year)

7.27124321326576e6*year

### Intractable?

After a couple weeks of hard work:
- $\approx$ 45 minutes
- $\approx$ 15 Gb
- 10 cores
- dimension: $587504$

In [4]:
%runfile code.py
%display unicode_art
tensor.symbol=u" ⊗ "
harmonic_bicharacter(6)

Compiling ./code1.pyx...


1 ⊗ s        + s   ⊗ s       + s   ⊗ s      + s   ⊗ s     + s   ⊗ s    + s   ⊗ 
     ┌┬┬┬┬┬┐    ┌┐    ┌┬┬┬┬┐    ┌┐    ┌┬┬┬┐    ┌┐    ┌┬┬┐    ┌┐    ┌┬┐    ┌┐   
     └┴┴┴┴┴┘    └┘    ├┼┴┴┴┘    ├┤    ├┼┴┴┘    ├┤    ├┼┴┘    ├┤    ├┼┘    ├┤   
                      └┘        └┘    ├┤       ├┤    ├┤      ├┤    ├┤     ├┤   
                                      └┘       └┘    ├┤      ├┤    ├┤     ├┤   
                                                     └┘      └┘    ├┤     ├┤   
                                                                   └┘     └┘   
                                                                               

s   + s    ⊗ s      + s    ⊗ s       + s    ⊗ s     + s    ⊗ s      + s    ⊗ 
 ┌┐    ┌┬┐    ┌┬┬┬┐    ┌┬┐    ┌┬┬┬┬┐    ┌┬┐    ┌┬┬┐    ┌┬┐    ┌┬┬┬┐    ┌┬┐   
 ├┤    └┴┘    ├┼┼┴┘    └┴┘    ├┼┴┴┴┘    ├┼┘    ├┼┼┘    ├┼┘    ├┼┴┴┘    ├┼┘   
 ├┤           └┴┘             └┘        └┘     ├┼┘     └┘     ├┤       └┘    
 ├┤                                            

## Was it worth the trouble?

**Stability property**: $\harm(6,5)$ contains all the information about $\harm(6,r)$ for all $r$.

«Coté mathématiques, grâce à tes calculs et à mon apprentissage de Sage, j’ai exploré une mine d’or de liens entre tout ce qu’on fait depuis le début sur les opérateurs de Macdonald, jusqu’à aujourd’hui; algèbre de Hall elliptique, Conjecture Delta, théorie des noeuds et  entrelacs du tore, etc. Je pense que cela sera ce que j’ai fait de mieux dans ma carrière. C’est magnifique et cela simplifie la compréhension de tout, permet de prouver plein de liens nouveaux, prédit comment généraliser, et de plus j’ai des modules qui expliquent cela.»

François

## Plan
### Motivations
### Warming up: harmonic polynomials
- Definition
- Exploiting the grading

### Diagonal harmonic polynomials
- Definition
- Exploiting the multigrading
- Exploiting symmetries
- Exploiting the action of the symmetric group `S_n`
- Exploiting antisymmetries
- Exploiting the action of `gl_k`

## Harmonic polynomials
$\QQ[\x]$: the ring of polynomials in the $n$ variables $X:=(x_1,\ldots,x_n)$

**Definition:** $f\in\QQ[X]$ **harmonic** if
$$ \frac{\partial^k}{\partial x_1} f+ \cdots + \frac{\partial^k}{\partial x_n} f = 0, \qquad \forall k>0$$

In particular: $\quad \div f = 0, \quad  \nabla f = 0, \quad ...$

**Example**: $1$, $x_1-x_2$ and linear combinations are harmonic.

### Notations
- $\harm(n)$ : subspace of harmonic polynomials
- $\partial_i := \frac{\partial}{\partial x_i}$

$\harm(n)$ : joint kernel of the differential operators $\partial_1^k + \cdots + \partial_n^k$

$\harm(n)$ : a realization of the coinvariants of the symmetric group: $\QQ[X] / \langle \QQ[X]^{S_n} \rangle$ 

### Example
$$\harm(2) = \langle 1, x_1-x_2 \rangle_\QQ$$

### Example on computer

In [5]:
%display latex
R = QQ['x1,x2,x3']
x1,x2,x3 = X = R.gens()
p1 = lambda p: p.derivative(x1)                         # ∂₁
p2 = lambda p: p.derivative(x2)                         # ∂₂
p3 = lambda p: p.derivative(x3)                         # ∂₃
D1 = lambda p: sum( p.derivative(v) for v in X)         # ∂₁  + ∂₂  + ∂₃
D2 = lambda p: sum( p.derivative([v,v]) for v in X)     # ∂₁² + ∂₂² + ∂₃²
D3 = lambda p: sum( p.derivative([v,v,v]) for v in X)   # ∂₁³ + ∂₂³ + ∂₃³

In [6]:
D1(x1-x2)

0

In [7]:
D1(x1-x3), D1(x2-x3)

(0, 0)

In [8]:
D1(x1^2-x2^2)

2*x1 - 2*x2

In [9]:
Delta = ( (x1-x2) * (x1-x3) * (x2-x3) )
D1(Delta), D2(Delta), D3(Delta)

(0, 0, 0)

In [10]:
f = p1(Delta)
D1(f), D2(f), D3(f)

(0, 0, 0)

### What have we learned?
- $\Delta:=$ $\prod_{i<j}(x_i-x_j)$ is harmonic
- $f$ harmonic $\Longrightarrow$ $\partial_i f$ harmonic

### Theorem
$$\harm(n) = \langle \Delta \rangle_{\partial_1,\ldots,\partial_n}$$

### On computer

In [11]:
%runfile code.py
H = Subspace([Delta], [p1,p2,p3])
H.dimension()

6

In [12]:
H.basis()

(-x1^2*x2 + x1*x2^2 + x1^2*x3 - x2^2*x3 - x1*x3^2 + x2*x3^2,
 -x1*x2 + 1/2*x2^2 + x1*x3 - 1/2*x3^2,
 1/2*x1^2 - x1*x2 + x2*x3 - 1/2*x3^2,
 x1 - x3,
 x2 - x3,
 1)

In [13]:
[factor(v) for v in H.basis()]

[(x2 - x3) * (-x1 + x2) * (x1 - x3),
 (1/2) * (x2 - x3) * (-2*x1 + x2 + x3),
 (-1/2) * (-x1 + 2*x2 - x3) * (x1 - x3),
 x1 - x3,
 x2 - x3,
 1]

### Algorithm: straightforward linear algebra in the monomial basis
`Subspace( polynomials, operators )`:
- maintain a list $L$ of all the relevant monomials
- represent the polynomials as finite vectors with columns indexed by $L$
- build a matrix and maintain it in row echelon form
- insert new vectors until the subspace is stable under the operators

**Tricky part**: make that work with most vector spaces in Sage

In [14]:
H.matrix()

[   1   -1    1    1   -1   -1    0    0    0    0    0    0    0    0    0    0]
[   0    0    0    0    0    0    1    0    0  1/2    0    0    0    0 -1/2   -1]
[   0    0    0    0    0    0    0    1  1/2    0    0    0    0    0 -1/2   -1]
[   0    0    0    0    0    0    0    0    0    0    1    0    0   -1    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    1    0   -1    0    0]
[   0    0    0    0    0    0    0    0    0    0    0    0    1    0    0    0]

**Note**: The matrix is block diagonal: harmonic polynomials of degree $0, 1, 2, 3$

### Strategy: exploiting the grading

In [15]:
def add_degrees(d1, d2):
    d = d1 + d2
    if d < 0: raise ValueError("Negative degree")
    return d
F = Subspace(generators  = { 3: [Delta]},
             operators   = {-1: [p1,p2,p3]},
             add_degrees = add_degrees)

In [16]:
F.dimension()

6

In [17]:
F.hilbert_polynomial()

q^3 + 2*q^2 + 2*q + 1

**Algorithm**:
- construct the graph of the homogeneous components and operators between them
- maintain one matrix per homogeneous component and propagate polynomials

**Potential gain** for an $n\times n$ matrix $M$ with $k$ blocks of same size:<br>
Complexity of linear algebra: $k\left(\frac n d\right)^3 = \frac 1{k^2} n^3$

So far, nothing very interesting, since $\harm(n,1)$ is very well known.
Time to move on to diagonal harmonic polynomials

## Diagonal harmonic polynomials
Polynomial ring in $r$ rows $X_i$ of $n$ variables $\QQ\begin{bmatrix}
        x_{11}&\cdots&x_{1n}\\
        \vdots & & \vdots\\
        x_{r1}&\cdots&x_{rn}\\
     \end{bmatrix}$

Draw the matrix of variables on the board, with $X$, $X_1$, ...

Differential operators $(D_\alpha)_{\alpha\in \NN^r}$:
$$D_{(3,0,4)} := \partial_{1,1}^3\partial_{3,1}^4 + \dots + \partial_{1,n}^3\partial_{3,n}^4$$

**Diagonal harmonic polynomials** $\harm(n,r)$: joint kernel of the $(D_\alpha)_\alpha$

How to compute them?

## Computing diagonal harmonic polynomials

**Remark**: $\harm(n,r)$ contains $\harm(n,1)$

**Polarization operators:**<br>
In two rows $X_1=(x_1,\ldots,x_n)$ and $X_2=(y_1,\ldots,y_n)$ of variables:
$$P_{(-k,1)} := \partial_1^k y_1 +\cdots+\partial_n^k y_n$$
Between any two rows $X_i$ and $X_j$ of variables:
$$P_{(\dots,-k,\dots,1,\dots)} := \cdots$$ 

**Theorem:**
$\harm(n,r)$ is generated by $\harm(n,1)$ and the polarization operators.

## Multigrading
**Multidegree**: degree in each row $X_i$ of variables

**Example**: $x_{1,1}^2x_{1,2} x_{3,2}^4$ is of multidegree $(3,0,4)$

**Remark**:
- The $D_\alpha$ preserve the grading: $\harm(n,r)$ is multigraded
- The polarization operators preserve the multigrading
- So we can split the linear algebra by homogeneous components

Draw on the board
TODO: do the calculation directly with Subspace and polarization operators

Example: computing the **Hilbert polynomial** of Harm(3,2):

In [42]:
n = 3; r = 2
sum(  harmonic_character(mu).expand(r, 'q') * StandardTableaux(mu).cardinality()
    for mu in Partitions(n))

q0^3 + q0^2*q1 + q0*q1^2 + q1^3 + 2*q0^2 + 3*q0*q1 + 2*q1^2 + 2*q0 + 2*q1 + 1

In [43]:
sum(_.coefficients())

16

TODO: Could show the matrices; or later, when illustrating antisymmetries

## Symmetries

In [38]:
n = 3; r = 2
sum(harmonic_character(mu).expand(r, 'q') * StandardTableaux(mu).cardinality()
    for mu in Partitions(n))

q0^3 + q0^2*q1 + q0*q1^2 + q1^3 + 2*q0^2 + 3*q0*q1 + 2*q1^2 + 2*q0 + 2*q1 + 1

In [41]:
n = 4; r = 3
sum(harmonic_character(mu) * StandardTableaux(mu).cardinality()
    for mu in Partitions(n))


s[] + 3*s[1] + 3*s[1, 1] + s[1, 1, 1] + 5*s[2] + 5*s[2, 1] + 6*s[3] + 4*s[3, 1] + 5*s[4] + s[4, 1] + 3*s[5] + s[6]

How comes?

# Symmetries
$\harm(n,r)$ stable under:

- Permuting columns of variables: action of the symmetric group $S_n$

- Permuting rows of variables: action of the symmetric group $S_r$<br>
  $\Longrightarrow$ symmetry in the multidegrees!

- Linear combinations of rows: acting of the general linear group $GL(r)$<br>
  The homogeneous components are the weight spaces<br>
  The symmetric function is the $GL(r)$-character of $\harm(n,r)$

In [ ]:
Remember: $\Delta$ is harmonic because of antisymmetries

- Permuting rows of variables: action of the symmetric group $S_r$

## Notations

- $\x^\alpha := x_1^{a_1}\cdots x_n^{a_n}$
- $\partial_\x^\alpha := \partial_i^{a_1} \cdots \partial_n^{a_n}$